# System Messages Examples

This notebook demonstrates the usage of the SystemMessage and SystemMessageManager classes for managing processing instructions.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
from pathlib import Path
from tnh_scholar.openai_text_processing import SystemMessage, SystemMessageManager, TaskType, ContentType

# Set your system messages directory here
system_messages_dir = Path("./system_messages")
manager = SystemMessageManager(system_messages_dir)

## 1. Creating Basic Messages

Let's create some system messages for different tasks.

In [3]:
# Create a formatting message for Dharma talks
format_msg = SystemMessage(
    task_type=TaskType.FORMAT,
    content_type=ContentType.DHARMA_TALK,
    input_language="en",
    instructions="""You are formatting a section entitled '{section_title}' from a Dharma Talk 
    offered by {speaker_name}. Format the text into meaningful paragraphs while 
    correcting any transcription or grammatical errors.""",
    template_fields={"section_title": "", "speaker_name": "Unknown Speaker"},
    description="Format Dharma talk sections with corrections",
    tags=["plum_village", "transcription", "formatting"]
)

# Save the message
manager.save_message(format_msg)

print(f"Message ID: {format_msg.msg_id}")

Message ID: format_dharma_talk_en_default_2024-12-26T22:38:39.792279


In [5]:
# Create a translation message
translate_msg = SystemMessage(
    task_type=TaskType.TRANSLATE,
    content_type=ContentType.ARTICLE,
    input_language="vi",  # Vietnamese
    created_at=datetime.now(),
    updated_at=datetime.now(),
    instructions="""Translate the following {content_type} from Vietnamese to English. 
    Maintain the original meaning and style while ensuring natural English flow. 
    Context: {context}""",
    template_fields={"content_type": "article", "context": ""},
    description="Vietnamese to English translation for articles",
    tags=["translation", "vietnamese"]
)

manager.save_message(translate_msg)

## 2. Applying Templates

Let's see how to use the template system with different values.

In [6]:
# Apply template with specific values
formatted_instructions = manager.apply_template(
    format_msg,
    {
        "section_title": "Mindfulness in Daily Life",
        "speaker_name": "Sister Chan Khong"
    }
)

print("Formatted instructions:")
print(formatted_instructions)

Formatted instructions:
You are formatting a section entitled 'Mindfulness in Daily Life' from a Dharma Talk 
    offered by Sister Chan Khong. Format the text into meaningful paragraphs while 
    correcting any transcription or grammatical errors.


## 3. Updating Messages

Let's create some updates and see the version history.

In [7]:
# Update the formatting message with new instructions
updated_msg = manager.update_message(
    task_type=TaskType.FORMAT,
    content_type=ContentType.DHARMA_TALK,
    input_language="en",
    instructions="""You are formatting a section entitled '{section_title}' from a Dharma Talk 
    offered by {speaker_name}. Format the text into meaningful paragraphs while 
    correcting any transcription or grammatical errors. Use <p> tags for paragraphs.""",
    tags=["plum_village", "transcription", "formatting", "html"]
)

# Get the history of all versions
history = manager.get_message_history(
    TaskType.FORMAT,
    ContentType.DHARMA_TALK,
    "en"
)

print(f"Number of versions: {len(history)}")
for msg in history:
    print(f"\nVersion from {msg.updated_at}:")
    print(f"Instructions: {msg.instructions[:100]}...")
    print(f"Tags: {msg.tags}")

Number of versions: 2

Version from 2024-12-26 16:28:25.776406:
Instructions: You are formatting a section entitled '{section_title}' from a Dharma Talk 
    offered by {speaker_...
Tags: ['plum_village', 'transcription', 'formatting']

Version from 2024-12-26 16:29:42.367006:
Instructions: You are formatting a section entitled '{section_title}' from a Dharma Talk 
    offered by {speaker_...
Tags: ['plum_village', 'transcription', 'formatting', 'html']


## 4. Complex Example: Multiple Updates and Templates

Let's create a more complex workflow with multiple updates and template applications.

In [7]:
# Create a sectioning message for a book
section_msg = SystemMessage(
    task_type=TaskType.SECTION,
    content_type=ContentType.BOOK,
    input_language="en",
    created_at=datetime.now(),
    updated_at=datetime.now(),
    instructions="""Process '{book_title}' by {author}. 
    Split the text into chapters based on {delimiter}. 
    Each chapter should start with '{chapter_prefix}'.
    Context: {context}""",
    template_fields={
        "book_title": "",
        "author": "Unknown Author",
        "delimiter": "Chapter",
        "chapter_prefix": "Chapter",
        "context": ""
    },
    description="Section books into chapters",
    tags=["book", "sectioning"]
)

manager.save_message(section_msg)

# Make several updates
updates = [
    {"description": "Updated section format for books", 
     "template_fields": {
         "book_title": "",
         "author": "Unknown Author",
         "delimiter": "Chapter|Part",  # Added Part as delimiter
         "chapter_prefix": "Chapter",
         "context": ""
     }},
    {"instructions": """Process '{book_title}' by {author}. 
    Split the text into chapters based on {delimiter}. 
    Each chapter should start with '{chapter_prefix}'.
    Add <chapter> tags around each chapter.
    Context: {context}"""},
    {"tags": ["book", "sectioning", "xml"]}
]

for update in updates:
    section_msg = manager.update_message(
        TaskType.SECTION,
        ContentType.BOOK,
        "en",
        **update
    )

# Get all versions and show their differences
history = manager.get_message_history(TaskType.SECTION, ContentType.BOOK, "en")

print(f"Evolution of the sectioning message ({len(history)} versions):")
for i, msg in enumerate(history, 1):
    print(f"\nVersion {i} ({msg.updated_at}):")
    print(f"Tags: {msg.tags}")
    print(f"Template fields: {msg.template_fields}")
    print("Instructions preview: ", msg.instructions[:100], "...")

Evolution of the sectioning message (4 versions):

Version 1 (2024-12-26 15:26:10.274364):
Tags: ['book', 'sectioning']
Template fields: {'book_title': '', 'author': 'Unknown Author', 'delimiter': 'Chapter', 'chapter_prefix': 'Chapter', 'context': ''}
Instructions preview:  Process '{book_title}' by {author}. 
    Split the text into chapters based on {delimiter}. 
    Eac ...

Version 2 (2024-12-26 15:26:10.278525):
Tags: ['book', 'sectioning']
Template fields: {'book_title': '', 'author': 'Unknown Author', 'delimiter': 'Chapter|Part', 'chapter_prefix': 'Chapter', 'context': ''}
Instructions preview:  Process '{book_title}' by {author}. 
    Split the text into chapters based on {delimiter}. 
    Eac ...

Version 3 (2024-12-26 15:26:10.279902):
Tags: ['book', 'sectioning']
Template fields: {'book_title': '', 'author': 'Unknown Author', 'delimiter': 'Chapter|Part', 'chapter_prefix': 'Chapter', 'context': ''}
Instructions preview:  Process '{book_title}' by {author}. 
    Split the text

## 5. Querying Messages

In [ ]:
# Get latest versions of all format messages
format_messages = [
    manager.get_latest_message(TaskType.FORMAT, content_type, "en")
    for content_type in ContentType
]

print("Latest format messages:")
for msg in format_messages:
    if msg:  # Some content types might not have format messages
        print(f"\n{msg.content_type.value}:")
        print(f"Last updated: {msg.updated_at}")
        print(f"Template fields: {list(msg.template_fields.keys())}")

## 6. Error Handling Examples

Let's see how the system handles various error cases.

In [ ]:
try:
    # Try to create a message with mismatched template fields
    bad_msg = SystemMessage(
        task_type=TaskType.FORMAT,
        content_type=ContentType.ARTICLE,
        input_language="en",
        created_at=datetime.now(),
        updated_at=datetime.now(),
        instructions="Format the {title} with {author}",
        template_fields={"wrong_field": ""},  # Doesn't match instructions
        description="This should fail"
    )
    manager.save_message(bad_msg)
except ValueError as e:
    print("Expected error caught:")
    print(e)

try:
    # Try to update a non-existent message
    result = manager.update_message(
        TaskType.FORMAT,
        ContentType.NEWSLETTER,
        "fr",  # No French messages exist
        instructions="This should fail"
    )
    if result is None:
        print("\nUpdate failed as expected: No message found to update")
except Exception as e:
    print(f"\nUnexpected error: {e}")